In [109]:
import time
import math
import re
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from pandas_ods_reader import read_ods
from  matplotlib.ticker import PercentFormatter
import seaborn as sns
 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
 
from sklearn import set_config
# set_config(display='diagram') # Để trực quan hóa pipeline
 
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.utils import resample

## Combine

In [4]:
df_1 = pd.read_excel('data/RTAs_2012_2018_Time_Series.xlsx')

In [5]:
df_2 = pd.read_excel('data/Road Safety Data - Accidents 2019.xlsx')

In [6]:
df_1.shape

(959847, 32)

In [10]:
df_1 = df_1.append(df_2,ignore_index = True)

In [13]:
df_1.to_csv('data/RTAs_2012_2019_Time_Series.csv',index=False)

## Preprocessing

https://roadtraffic.dft.gov.uk/api/local-authorities/

In [113]:
df = pd.read_csv('data/RTAs_2012_2019_Time_Series.csv')
df.shape

(1077383, 32)

In [114]:
df.columns

Index(['Accident_Index', 'Location_Easting_OSGR', 'Location_Northing_OSGR',
       'Longitude', 'Latitude', 'Police_Force', 'Accident_Severity',
       'Number_of_Vehicles', 'Number_of_Casualties', 'Date', 'Day_of_Week',
       'Time', 'Local_Authority_(District)', 'Local_Authority_(Highway)',
       '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit',
       'Junction_Detail', 'Junction_Control', '2nd_Road_Class',
       '2nd_Road_Number', 'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area', 'Did_Police_Officer_Attend_Scene_of_Accident',
       'LSOA_of_Accident_Location'],
      dtype='object')

Ta bỏ bớt một số cột không cần dùng trong đồ án này

In [119]:
df = df.drop(['Accident_Index','Location_Easting_OSGR', 'Location_Northing_OSGR','Longitude', 'Latitude', 'Police_Force', 'Local_Authority_(District)', '1st_Road_Number', '2nd_Road_Number', 'Did_Police_Officer_Attend_Scene_of_Accident',
       'LSOA_of_Accident_Location'],1)

In [1]:
# df.loc(lambda df: df['Date'] == '2012-01-01 00:00:00')

In [154]:
df['Date'].unique()

array(['2012-01-01 00:00:00', '2012-01-02 00:00:00',
       '2012-01-03 00:00:00', ..., '31/08/2019', '31/10/2019',
       '31/12/2019'], dtype=object)

In [152]:
for date in df['Date'].unique():
    df[df['Date'] == date]['Date'] = date[0:10]


<ipython-input-152-84dac9fc18ba>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Date'] == date]['Date'] = date[0:10]


In [139]:
df['Date'] + ' ' + df['Time']

0          2012-01-01 00:00:00 00:04:00
1          2012-01-01 00:00:00 00:05:00
2          2012-01-01 00:00:00 00:05:00
3          2012-01-01 00:00:00 00:09:00
4          2012-01-01 00:00:00 00:09:00
                       ...             
1077378             31/12/2019 23:42:00
1077379             31/12/2019 23:45:00
1077380             31/12/2019 23:45:00
1077381             31/12/2019 23:50:00
1077382             31/12/2019 23:53:00
Length: 1077383, dtype: object

In [120]:
df.columns

Index(['Accident_Severity', 'Number_of_Vehicles', 'Number_of_Casualties',
       'Date', 'Day_of_Week', 'Time', 'Local_Authority_(Highway)',
       '1st_Road_Class', 'Road_Type', 'Speed_limit', 'Junction_Detail',
       'Junction_Control', '2nd_Road_Class',
       'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area'],
      dtype='object')

Hiện tại một số cột đang có dạng số, nhưng thực ra là dạng categorical không có thứ tự, ta chuyển lại về chữ nhờ file `variable_lookup.xls` do chính phủ Anh cung cấp để dễ dàng hiểu dữ liệu và tìm insight hơn.

Các cột này là: 'Accident_Severity', 'Local_Authority_(Highway)', '1st_Road_Class', 'Road_Type', 'Junction_Detail', 'Junction_Control', '2nd_Road_Class', 'Pedestrian_Crossing-Human_Control', 'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions', 'Weather_Conditions', 'Road_Surface_Conditions', 'Special_Conditions_at_Site', 'Carriageway_Hazards', 

Các cột dạng categorical có thứ tự: ta chuyển lại về chữ cho dataframe dùng để tìm insight, nhưng giữ nguyên cho dataframe dùng để huấn luyện model:

'Day_of_Week', 'Urban_or_Rural_Area'

Tuy nhiên, ta thấy cột 'Local_Authority_(Highway)' có đến 208 giá trị, ta sẽ chuyển các vùng địa phương này thành khu vực (region) nó phụ thuộc vào, để nghiến cứu trên phạm vi lớn hơn, ta có thể chuyển nhờ API sau:

https://roadtraffic.dft.gov.uk/api/local-authorities

In [127]:
unordered_cate_cols = ['Road_Type', 'Junction_Detail', 
                'Junction_Control', 'Pedestrian_Crossing-Human_Control', 
                'Pedestrian_Crossing-Physical_Facilities',
                'Special_Conditions_at_Site', 'Carriageway_Hazards']
        
ordered_cate_cols = ['Accident_Severity', 'Day_of_Week', '1st_Road_Class',
                             '2nd_Road_Class', 'Light_Conditions', 'Weather_Conditions',
                             'Road_Surface_Conditions', 'Urban_or_Rural_Area']

In [111]:
class ColMap(BaseEstimator, TransformerMixin):
    def __init__(self, mode = 'insight'):
#         mode = `insight` or `trainning`
        self.mode = mode
        pass
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df, y=None):
        tf_df = X_df.copy()
        
        # Bước 1. Map
        
        if self.mode == 'trainning' or self.mode == 'insight':
            
            for col in unordered_cate_cols:
                if col == 'Pedestrian_Crossing-Human_Control':
                    name = 'Ped Cross - Human'
                elif col == 'Pedestrian_Crossing-Physical_Facilities':
                    name = 'Ped Cross - Physical'
                elif col == 'Urban_or_Rural_Area':
                    name = 'Urban Rural'
                else:
                    name = col.replace('_',' ')
                    
                map_df = pd.read_excel('data/variable_lookup.xls',name)
                for code in map_df['code']:
                    tf_df[col] = tf_df[col].replace(code,map_df[map_df['code'] == code]['label'].values[0])
                    
        if self.mode == 'insight':
            for col in ordered_cate_cols:
                if col == 'Weather_Conditions':
                    name = 'Weather'
                elif col == 'Road_Surface_Conditions':
                    name = 'Road Surface'
                else:
                    name = col.replace('_',' ')
                    
                map_df = pd.read_excel('data/variable_lookup.xls',name)
                for code in map_df['code']:
                    tf_df[col] = tf_df[col].replace(code,map_df[map_df['code'] == code]['label'].values[0])
                    
        # Bước 2. Xử lý date time
        
        #if Ons code has not exist yet
        tf_df['Ons code'] = np.nan
        ##
        
        tf_df = tf_df.drop(['Ons code', 'Accident severity', 'Accident year'], axis=1)
        
        # Bước 2
        tf_df['Junction type'] = tf_df['Junction type'].replace('Unknown', np.nan)
        tf_df['Speed limit'] = tf_df['Speed limit'].replace('Unknown', np.nan)
        
        # Bước 3
        tf_df = tf_df.dropna(axis = 0, thresh = 3)
        tf_df = tf_df.dropna(axis = 1, how = 'all')
        return tf_df

In [121]:
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]

In [56]:
contigency_df = pd.crosstab(df_1['Local_Authority_(District)'], df_1['Accident_Severity'])
contigency_df.columns = ['1','2','3']

In [97]:
print((sum(contigency_df['1']) + sum(contigency_df['2']) + sum(contigency_df['3'])) == df_1.shape[0])
contigency_df
# df_1.shape
# contigency_df.xs('Accident_Severity', level='1', axis=1)
# items_df_preprocessed[df_1['Region'] == f'{region}'].groupby('Accident severity')['Accidents'].agg(sum).reset_index(name='Value count')

True


,1,2,3
Local_Authority_(District),,,
1,51,1526,10916
2,32,845,6132
3,16,806,5568
4,37,873,6082
5,40,971,7496
...,...,...,...
937,33,276,1080
938,77,542,2829
939,39,335,1008


In [ ]:
Region_Severity_df = pd.DataFrame({'Region' : [np.nan],'Fatal' : [np.nan],'Serious' : [np.nan],'Slight' : [np.nan], 'Pr(Ai)': [np.nan]})

for region in df_1['LSOA_of_Accident_Location'].unique():
    x = items_df_preprocessed[df_1['Region'] == f'{region}'].groupby('Accident severity')['Accidents'].agg(sum).reset_index(name='Value count')
    y = np.array(x['Value count'])
  # y = np.insert(y, 0, 3, axis=0)
  # source: https://stackoverflow.com/a/58293212
    data_to_append = {}
    for i in range(len(Region_Severity_df.columns)):
        if(i>=1 and i !=4):
            data_to_append[Region_Severity_df.columns[i]] = y[i-1]
        elif (i == 4):
            data_to_append[Region_Severity_df.columns[i]] = sum(y)
            
    Region_Severity_df = Region_Severity_df.append(data_to_append, ignore_index = True)

Region_Severity_df = Region_Severity_df.drop(0,0)
Region_Severity_df['Region'] = RTAs_region_year_df['Region'].unique()
print(sum(Region_Severity_df['Pr(Ai)']))
sum_accidents = sum(Region_Severity_df['Pr(Ai)'])
Region_Severity_df